In [0]:
# Instalar paho-mqtt y la biblioteca de Azure Storage
#%pip install paho-mqtt azure-storage-blob


  Obtaining dependency information for paho-mqtt from https://files.pythonhosted.org/packages/c4/cb/00451c3cf31790287768bb12c6bec834f5d292eaf3022afc88e14b8afc94/paho_mqtt-2.1.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/67.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import paho.mqtt.client as mqtt
from azure.storage.blob import BlobServiceClient
from pyspark.sql import SparkSession

# Inicializar una lista para almacenar los mensajes
mqtt_messages = []

# Función que maneja los mensajes MQTT
def on_message(client, userdata, message):
    print(f"Mensaje recibido en el tópico {message.topic}: {message.payload.decode()}")
    mqtt_messages.append((message.topic, message.payload.decode()))  # Guardar en lista

# Configurar el cliente MQTT
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Conexión exitosa al broker MQTT")
        client.subscribe("#")  # Suscribirse a todos los tópicos
    else:
        print(f"Error al conectarse al broker MQTT. Código de error: {rc}")

mqtt_client = mqtt.Client()
mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

# Establecer el usuario y la contraseña
username = "user_abv"
password = "Tios1991m"
mqtt_client.username_pw_set(username, password)

# Conectar al broker MQTT
broker_address = "0.tcp.eu.ngrok.io"
mqtt_client.connect(broker_address, 17966, 60)





/root/.ipykernel/2415/command-773880048719815-2944179060:21: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [0]:
# Iniciar el loop para recibir mensajes en segundo plano
mqtt_client.loop_start()

# Esperar unos segundos para recibir mensajes
import time
time.sleep(5*60)  # Ajusta el tiempo según sea necesario

# Detener el loop del cliente MQTT
mqtt_client.loop_stop()

Conexión exitosa al broker MQTT
Mensaje recibido en el tópico ABV/huerto/tomate1: {"Zona":"Tomate1","Hora":"2024-09-17 19:00","T_amb":23.60000038,"HR_amb":62.29999924,"HR_soil":46,"irrad":103.9631424}
Mensaje recibido en el tópico ABV/huerto/tomate1: {"Zona":"Tomate1","Hora":"2024-09-17 19:01","T_amb":23.60000038,"HR_amb":62.09999847,"HR_soil":46,"irrad":104.3318024}
Mensaje recibido en el tópico ABV/huerto/tomate1: {"Zona":"Tomate1","Hora":"2024-09-17 19:02","T_amb":23.60000038,"HR_amb":62.29999924,"HR_soil":46,"irrad":103.5944748}
Mensaje recibido en el tópico ABV/huerto/tomate1: {"Zona":"Tomate1","Hora":"2024-09-17 19:03","T_amb":23.60000038,"HR_amb":62.20000076,"HR_soil":46,"irrad":104.5161285}
Mensaje recibido en el tópico ABV/huerto/tomate1: {"Zona":"Tomate1","Hora":"2024-09-17 19:04","T_amb":23.60000038,"HR_amb":62.29999924,"HR_soil":46,"irrad":103.9631424}


<MQTTErrorCode.MQTT_ERR_SUCCESS: 0>

In [0]:
# Crear un Spark DataFrame a partir de los mensajes recibidos
spark = SparkSession.builder.appName("MQTT-Databricks").getOrCreate()
rdd = spark.sparkContext.parallelize(mqtt_messages)
df = rdd.toDF(["topic", "message"])

# Mostrar los datos recibidos
df.show()

+------------------+--------------------+
|             topic|             message|
+------------------+--------------------+
|ABV/huerto/tomate1|{"Zona":"Tomate1"...|
|ABV/huerto/tomate1|{"Zona":"Tomate1"...|
|ABV/huerto/tomate1|{"Zona":"Tomate1"...|
|ABV/huerto/tomate1|{"Zona":"Tomate1"...|
|ABV/huerto/tomate1|{"Zona":"Tomate1"...|
+------------------+--------------------+



In [0]:
container = "datos"
account = spark.conf.get("adls.account.name")
datos_path = f"abfss://{container}@{account}.dfs.core.windows.net/datos-arduino"

In [0]:
print(datos_path)

abfss://datos@masterabv003sta.dfs.core.windows.net/datos-arduino


In [0]:
import json

# Función para leer datos del DataFrame en lugar de generarlos aleatoriamente
def leer_datos_sensor(df):
    # Inicializar una lista para almacenar los datos del sensor
    datos_sensor = []

    # Iterar sobre las filas del DataFrame para extraer los datos
    for row in df.collect():  # Recuerda que 'df' es un DataFrame de Spark, así que usamos 'collect()' para obtener las filas
        message = row['message']
        data = json.loads(message)  # Convertir el mensaje JSON a un diccionario
        
        # Extraer los valores del diccionario
        zona = data.get("Zona")
        hora = data.get("Hora")
        temperatura = data.get("T_amb")
        humedad_ambiente = data.get("HR_amb")
        humedad_suelo = data.get("HR_soil")
        radiacion = data.get("irrad")

        # Crear una tupla con los datos y añadirla a la lista
        datos_sensor.append((zona, hora, temperatura, humedad_ambiente, humedad_suelo, radiacion))
    
    # Convertir la lista en un DataFrame
    df_extract = spark.createDataFrame(datos_sensor, ["Zona", "DateTime", "T_amb", "HR_amb", "HR_soil", "irrad"])

    # Retornar la lista de datos del sensor
    return df_extract
    
df_extract = leer_datos_sensor(df)

df_extract.show()
  

+-------+----------------+-----------+-----------+-------+-----------+
|   Zona|        DateTime|      T_amb|     HR_amb|HR_soil|      irrad|
+-------+----------------+-----------+-----------+-------+-----------+
|Tomate1|2024-09-17 19:00|23.60000038|62.29999924|     46|103.9631424|
|Tomate1|2024-09-17 19:01|23.60000038|62.09999847|     46|104.3318024|
|Tomate1|2024-09-17 19:02|23.60000038|62.29999924|     46|103.5944748|
|Tomate1|2024-09-17 19:03|23.60000038|62.20000076|     46|104.5161285|
|Tomate1|2024-09-17 19:04|23.60000038|62.29999924|     46|103.9631424|
+-------+----------------+-----------+-----------+-------+-----------+



In [0]:
def guardar_datos(df, datos_path):
        df = df.repartition(1)
        df.write.format('delta').mode("overwrite").save(f"{datos_path}")     

In [0]:
guardar_datos(df_extract, datos_path)